In [14]:
csc_h1.data[:2]

array([0.6931472, 0.6931472], dtype=float32)

In [ ]:
from illico.utils.ranking import _sort_csc_columns_inplace
from illico.utils.type import scipy_to_nb
import scanpy as sc
import anndata as ad

@njit(nogil=True, cache=False)
def _sort_csc_columns_inplace(csc_matrix) -> None:
    """Sort CSC columns in place.

    Args:
        csc_matrix (CSCMatrix): Input CSC matrix.
    """
    # _assert_is_csc(csc_matrix)
    for j in range(csc_matrix.shape[1]):
        csc_matrix.data[csc_matrix.indptr[j] : csc_matrix.indptr[j + 1]].sort()


h1 = ad.read_h5ad('../vcc_h1_904_genes.h5ad')
h1.X = h1.layers["csc"]
csc_h1 = scipy_to_nb(h1.X)
_sort_csc_columns_inplace(csc_h1);
%timeit -n 1 -r 1 _sort_csc_columns_inplace(csc_h1)
h1 = ad.read_h5ad('../vcc_h1_904_genes.h5ad')
h1.X = h1.layers["csc"]
csc_h1 = scipy_to_nb(h1.X)
%timeit -n 1 -r 1 _sort_csc_columns_inplace.__wrapped__(csc_h1)

# h1 = ad.read_h5ad('../vcc_h1_904_genes.h5ad')
# h1.X = h1.layers["csc"]
# sc.pp.normalize_total(h1, target_sum=1e4)
# csc_h1 = scipy_to_nb(h1.X.tocsc())
# # _sort_csc_columns_inplace(csc_h1);
# %timeit -n 1 -r 1 _sort_csc_columns_inplace(csc_h1)
# h1 = ad.read_h5ad('../vcc_h1_904_genes.h5ad')
# h1.X = h1.layers["csc"]
# sc.pp.normalize_total(h1, target_sum=1e4)
# csc_h1 = scipy_to_nb(h1.X.tocsc())
# %timeit -n 1 -r 1 _sort_csc_columns_inplace.__wrapped__(csc_h1)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
tuple index out of range
During: typing of static-get-item at /var/folders/y7/p3ldjm_s1xn8knz8bqf8f9p00000gn/T/ipykernel_3600/3742943492.py (14)

File "../../../../../../../var/folders/y7/p3ldjm_s1xn8knz8bqf8f9p00000gn/T/ipykernel_3600/3742943492.py", line 14:
<source missing, REPL/exec in use?>

During: Pass nopython_type_inference

In [31]:
import anndata as ad
import numpy as np
from illico.utils.type import scipy_to_nb
from numba import njit
import scanpy as sc

h1 = ad.read_h5ad('../vcc_h1_904_genes.h5ad')
h1.X = h1.layers["csc"]
# sc.pp.normalize_total(h1, target_sum=1e4)
# csc_h1 = scipy_to_nb(h1.X)
csc_h1 = h1.X
nb_argsort = njit(nogil=True)(lambda x: np.argsort(x))
nb_argsort(csc_h1.data[csc_h1.indptr[0]:csc_h1.indptr[1]])
%timeit -n 1 -r 1 [nb_argsort(csc_h1.data[csc_h1.indptr[i]:csc_h1.indptr[i+1]]) for i in range(csc_h1.shape[1])]
%timeit -n 1 -r 1 [np.argsort(csc_h1.data[csc_h1.indptr[i]:csc_h1.indptr[i+1]]) for i in range(csc_h1.shape[1])]
%timeit -n 1 -r 1 [csc_h1.data[csc_h1.indptr[i]:csc_h1.indptr[i+1]].sort() for i in range(csc_h1.shape[1])]

2.66 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
1.97 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
281 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [1]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler
from illico.asymptotic_wilcoxon import encode_and_count_groups
from illico.ovr.sparse_ovr import csc_ovr_mwu_kernel_over_contiguous_col_chunk
from illico.ovr import dense_ovr_mwu_kernel
from illico.utils.type import scipy_to_nb
import anndata as ad


h1 = ad.read_h5ad('../vcc_h1_904_genes.h5ad')
# X = scipy_to_nb(h1.layers["csc"])
# grpc = encode_and_count_groups(h1.obs['target_gene'].tolist(), ref_group=None)
# csc_ovr_mwu_kernel_over_contiguous_col_chunk(X, 0, 904, grpc, is_log1p=False);
# %lprun -f csc_ovr_mwu_kernel_over_contiguous_col_chunk csc_ovr_mwu_kernel_over_contiguous_col_chunk(X, 0, 904, grpc, is_log1p=False)
X = scipy_to_nb(h1.layers["dense"])
grpc = encode_and_count_groups(h1.obs['target_gene'].tolist(), ref_group=None)
dense_ovr_mwu_kernel(X, grpc, is_log1p=False);
%lprun -f dense_ovr_mwu_kernel dense_ovr_mwu_kernel(X, grpc, is_log1p=False)

Timer unit: 1e-09 s

Total time: 5.80105 s
File: /Users/remydubois/Documents/perso/repos/illico/illico/ovr/dense_ovr.py
Function: dense_ovr_mwu_kernel at line 14

Line #      Hits         Time  Per Hit   % Time  Line Contents
    14                                           def dense_ovr_mwu_kernel(
    15                                               X: np.ndarray,
    16                                               grpc: GroupContainer,
    17                                               use_continuity: bool = True,
    18                                               is_log1p: bool = False,
    19                                           ) -> tuple[np.ndarray]:
    20                                               """ Compute OVR ranksum test on a dense matrix of expression counts.
    21                                           
    22                                               Args:
    23                                                   X (np.ndarray): Input dense raw coun

In [ ]:
from scipy import sparse
h1_small = h1[:, :9].copy()
h1_small.layers["csr"] = h1_small.layers["csr"].copy()
h1_small.layers["dense"] = h1_small.layers["dense"].copy()
h1_small.layers["csc"] = h1_small.layers["csc"].copy()
del h1_small.X
h1_small.write('../vcc_h1_9_genes.h5ad')

In [11]:
h1_small = ad.read_h5ad('../vcc_h1_904_genes.h5ad')
from illico.asymptotic_wilcoxon import asymptotic_wilcoxon
h1_small.X = h1_small.layers["csc"]

In [ ]:
import anndata as ad
from illico.asymptotic_wilcoxon import asymptotic_wilcoxon

# h1 = ad.read_h5ad('/Users/remydubois/Documents/repos/vcc/data/competition_support_set/competition_train.h5')[:, :1808*2].copy()
h1 = ad.read_h5ad('/Users/remydubois/Documents/repos/vcc/data/competition_support_set/competition_train.h5')
# print(h1.shape)
# asymptotic_wilcoxon(h1[:, :1], is_log1p=False, group_keys="target_gene", reference_group="non-targeting", n_threads=1, batch_size=64)
# %timeit -n 1 -r 1 asymptotic_wilcoxon(h1, is_log1p=False, group_keys="target_gene", reference_group="non-targeting", n_threads=1, batch_size=64)
# %timeit -n 1 -r 1 asymptotic_wilcoxon(h1, is_log1p=False, group_keys="target_gene", reference_group="non-targeting", n_threads=2, batch_size=64)
# %timeit -n 1 -r 1 asymptotic_wilcoxon(h1, is_log1p=False, group_keys="target_gene", reference_group="non-targeting", n_threads=4, batch_size=64)
# %timeit -n 1 -r 1 asymptotic_wilcoxon(h1, is_log1p=False, group_keys="target_gene", reference_group="non-targeting", n_threads=8, batch_size=64)

  0%|          | 0/57 [00:00<?, ?it/s]

31.3 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


  0%|          | 0/57 [00:00<?, ?it/s]

17.3 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [7]:
from illico.utils.math import _warn_log1p

_warn_log1p(h1.X, is_log1p=False)

/Users/remydubois/Documents/perso/repos/illico/illico/utils/math.py:83: UserWarning: The data appears to be log1p transformed (non-integer values found). However, 'is_log1p' is set to False. Please ensure that this is intended, as it may affect the results.
  warnings.warn(


In [13]:
%load_ext line_profiler

%lprun -f asymptotic_wilcoxon asymptotic_wilcoxon(h1, is_log1p=False, group_keys="target_gene", reference_group="non-targeting", n_threads=2, batch_size=256)

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


  0%|          | 0/8 [00:00<?, ?it/s]

Timer unit: 1e-09 s

Total time: 10.2848 s
File: /Users/remydubois/Documents/perso/repos/illico/illico/asymptotic_wilcoxon.py
Function: asymptotic_wilcoxon at line 19

Line #      Hits         Time  Per Hit   % Time  Line Contents
    19                                           def asymptotic_wilcoxon(
    20                                               adata: ad.AnnData,
    21                                               is_log1p: bool,
    22                                               group_keys: str,
    23                                               reference_group: str | None = None,
    24                                               n_threads: int = 1,
    25                                               batch_size: int = 256,
    26                                           ):
    27                                               """"""
    28                                           
    29         1      11000.0  11000.0      0.0      X = adata.X
    30             

In [ ]:
from collections import namedtuple
from typing import Any

import numpy as np

GroupContainer = namedtuple(
    "GroupContainer",
    [
        "raw_groups",
        "ref_group",
        "encoded_groups",
        "unique_raw_groups",
        "counts",
        "indices",
        "indptr",
        "encoded_ref_group",
    ],
)


def encode_and_count_groups(groups: np.ndarray, ref_group: Any) -> GroupContainer:
    """Groups are encoded no matter what they contain, string, floats, integers, etc"""
    # sorted_groups = groups[group_indices]
    # diff = np.diff(sorted_groups)
    unique_groups, group_counts = np.unique(groups, return_counts=True)
    groups_mapping = {g: i for i, g in enumerate(unique_groups)}
    encoded_groups = np.array([groups_mapping[g] for g in groups])
    # This should sort them in the same order as the first np.unique
    group_indices = np.argsort(groups)
    group_indptr = np.cumsum(np.insert(group_counts, 0, 0))

    return GroupContainer(
        raw_groups=groups,
        ref_group=ref_group,
        encoded_groups=encoded_groups,
        unique_raw_groups=unique_groups,
        counts=group_counts,
        indices=group_indices,
        indptr=group_indptr,
        encoded_ref_group=-1 if ref_group is None else groups_mapping[ref_group], # Weirdly enough, this must be -1 and not None, otherwise Numba fails to compile various functions, especially branching
    )

groups = h1.obs.target_gene.values
%timeit encode_and_count_groups(groups, None)

65.9 ms ± 1.19 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [28]:
import numpy as np

def unique_with_counts(x):
    idxs = np.argsort(x.ravel())
    aux = x[idxs]
    mask = np.empty(aux.shape, dtype=bool)
    mask[0] = True
    mask[1:] = aux[1:] != aux[:-1]
    uniques = aux[mask]
    counts = np.diff(np.nonzero(mask)[0].tolist() + [len(aux)])
    return uniques, counts

groups = np.array(h1.obs.target_gene.values)
%timeit -n 5 -r 5 unique_with_counts(groups)
%timeit -n 5 -r 5 np.unique(groups, return_counts=True)
uniques, counts = unique_with_counts(groups)
_u, _c = np.unique(groups, return_counts=True)
np.testing.assert_allclose(counts, _c)
assert np.equal(uniques, _u).all() == True

48.9 ms ± 903 μs per loop (mean ± std. dev. of 5 runs, 5 loops each)
49.1 ms ± 461 μs per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [36]:
from sklearn.preprocessing import LabelEncoder

def f(groups, ref_group):
    encoded_groups = LabelEncoder().fit_transform(groups.ravel())
    idxs = np.argsort(encoded_groups.ravel())
    aux = encoded_groups[idxs]
    mask = np.empty(aux.shape, dtype=bool)
    mask[0] = True
    mask[1:] = aux[1:] != aux[:-1]
    uniques = aux[mask]
    counts = np.diff(np.nonzero(mask)[0].tolist() + [len(aux)])
    group_indptr = np.cumsum(np.insert(counts, 0, 0))
    return GroupContainer(
        raw_groups=groups,
        ref_group=ref_group,
        encoded_groups=encoded_groups,
        unique_raw_groups=groups[mask],
        counts=counts,
        indices=idxs,
        indptr=group_indptr,
        encoded_ref_group=-1
    )

le = LabelEncoder()
%timeit -n 5 -r 5 le.fit_transform(groups)
%timeit -n 5 -r 5 unique_with_counts(encoded_groups)
encoded_groups = le.fit_transform(groups)
unique, counts = unique_with_counts(encoded_groups)
%timeit -n 5 -r 5 f(groups, None)

12.2 ms ± 963 μs per loop (mean ± std. dev. of 5 runs, 5 loops each)
5.25 ms ± 206 μs per loop (mean ± std. dev. of 5 runs, 5 loops each)
17.8 ms ± 1.64 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


# Memory profiling

from 

In [1]:
import anndata as ad
import numpy as np
from illico.asymptotic_wilcoxon import asymptotic_wilcoxon
from pdex import parallel_differential_expression
import scanpy as sc

adata = ad.read_h5ad('/Users/remydubois/Documents/repos/vcc/data/competition_support_set/competition_train.h5')
# adata.X = adata.layers["csc"]

In [14]:
%load_ext memory_profiler

%memit -c parallel_differential_expression(adata, groupby_key="target_gene", reference="non-targeting")

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


INFO:pdex._single_cell:Auto-Detected log1p for dataset.
INFO:pdex._single_cell:Log1p status: True
INFO:pdex._single_cell:Precomputing masks for each target gene
Identifying target masks: 100%|██████████| 151/151 [00:00<00:00, 19080.53it/s]
INFO:pdex._single_cell:Precomputing variable indices for each feature
Identifying variable indices: 100%|██████████| 90/90 [00:00<00:00, 1955893.06it/s]
INFO:pdex._single_cell:Creating shared memory memory matrix for parallel computing
INFO:pdex._single_cell:Creating generator of all combinations: N=13590
INFO:pdex._single_cell:Creating generator of all batches: N=136
INFO:pdex._single_cell:Initializing parallel processing pool
INFO:pdex._single_cell:Processing batches
Processing batches: 100%|██████████| 136/136 [00:29<00:00,  4.68it/s]
INFO:pdex._single_cell:Flattening results
INFO:pdex._single_cell:Closing shared memory pool


peak memory: 1810.25 MiB, increment: 659.38 MiB


In [13]:
%memit -c parallel_differential_expression(adata, groupby_key="target_gene", reference="non-targeting", num_workers=2)

INFO:pdex._single_cell:Auto-Detected log1p for dataset.
INFO:pdex._single_cell:Log1p status: True
INFO:pdex._single_cell:Precomputing masks for each target gene
Identifying target masks: 100%|██████████| 151/151 [00:00<00:00, 19063.30it/s]
INFO:pdex._single_cell:Precomputing variable indices for each feature
Identifying variable indices: 100%|██████████| 90/90 [00:00<00:00, 1966080.00it/s]
INFO:pdex._single_cell:Creating shared memory memory matrix for parallel computing
INFO:pdex._single_cell:Creating generator of all combinations: N=13590
INFO:pdex._single_cell:Creating generator of all batches: N=136
INFO:pdex._single_cell:Initializing parallel processing pool
INFO:pdex._single_cell:Processing batches
Processing batches: 100%|██████████| 136/136 [00:16<00:00,  8.14it/s]
INFO:pdex._single_cell:Flattening results
INFO:pdex._single_cell:Closing shared memory pool


peak memory: 2302.41 MiB, increment: 1187.94 MiB


In [ ]:
%load_ext memory_profiler
%memit asymptotic_wilcoxon(adata, group_keys="target_gene", reference_group="non-targeting", is_log1p=False, n_threads=8, batch_size=256)
%memit asymptotic_wilcoxon(adata, group_keys="target_gene", reference_group="non-targeting", is_log1p=False, n_threads=8, batch_size=64)

%memit asymptotic_wilcoxon(adata, group_keys="target_gene", reference_group="non-targeting", is_log1p=False, n_threads=4, batch_size=256)

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


  0%|          | 0/71 [00:00<?, ?it/s]

peak memory: 10572.84 MiB, increment: 1235.31 MiB


  0%|          | 0/283 [00:00<?, ?it/s]

peak memory: 11838.17 MiB, increment: 3018.27 MiB


  0%|          | 0/71 [00:00<?, ?it/s]

peak memory: 11644.97 MiB, increment: 222.38 MiB


In [5]:
%memit asymptotic_wilcoxon(adata, group_keys="target_gene", reference_group="non-targeting", is_log1p=False, n_threads=8, batch_size=16)

  0%|          | 0/1130 [00:00<?, ?it/s]

peak memory: 11810.62 MiB, increment: 5840.67 MiB


In [8]:
import anndata as ad

h1 = ad.read_h5ad('/Users/remydubois/Documents/repos/vcc/data/competition_support_set/competition_train.h5')

In [24]:
(h1.obs.target_gene.value_counts() // 500 * 500)

rng = np.random.RandomState(0)

# Random gene-specific mean expression levels
gene_means = rng.uniform(0.1, 5, size=100)

# Sample Poisson counts
dense_counts = rng.poisson(gene_means, size=(10_000, 100)).astype(np.float32)
dense_counts

array([[4., 5., 6., ..., 0., 8., 1.],
       [1., 4., 2., ..., 0., 2., 0.],
       [3., 4., 1., ..., 0., 6., 0.],
       ...,
       [3., 2., 5., ..., 0., 7., 0.],
       [1., 3., 6., ..., 0., 3., 0.],
       [3., 8., 1., ..., 0., 3., 0.]], shape=(10000, 100), dtype=float32)

In [30]:
# Generate test file
# Group counts
group_counts = h1.obs.target_gene.value_counts()
group_counts.index = ["control"] + [f"pert_{i}" for i in range(1, len(group_counts))]
group_counts.to_csv('../tests/data/group_counts.csv')

# Compute sparsity
sparsity = h1.X.data.size / np.prod(h1.X.shape)

# Compute gene means
mus = np.asarray(h1.X.mean(0)).ravel()

In [ ]:
n_genes = mus.size
n_cells = 2_000
data_placeholder = np.empty(int(n_genes * n_genes * sparsity), dtype=np.float32)
indices_placeholder = np.empty(int(n_genes * n_genes * sparsity), dtype=np.int32)
for j in range(n_genes):
    gene_data = rng.poisson(mus[j], size=n_cells).astype(np.float32)
    non_zero_indices = np.nonzero(gene_data)[0]
    col_start = int(j * n_genes * sparsity)
    col_end = int((j + 1) * n_genes * sparsity)
    data_placeholder[col_start:col_start + non_zero_indices.size] = gene_data[non_zero_indices]
    indices_placeholder[col_start:col_start + non_zero_indices.size] = non_zero_indices

array([2.4487075e-04, 7.2207469e-01, 1.6719058e-01, ..., 6.2261920e-02,
       9.4395095e-01, 6.7493135e-01], shape=(18080,), dtype=float32)

In [40]:
rng.poisson(mus, size=(100_000, mus.size))

array([[0, 0, 0, ..., 0, 1, 1],
       [0, 1, 0, ..., 0, 2, 0],
       [0, 0, 0, ..., 0, 3, 2],
       ...,
       [0, 0, 0, ..., 0, 1, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 2, 0, ..., 1, 2, 0]], shape=(100000, 18080))

In [4]:
import numpy as np
x = np.random.normal(size=(150 * 1500 + 40_000))
%timeit -n 1 -r 1 np.sort(x)
%timeit -n 1 -r 1 [np.sort(x[i:i+1500]) for i in range(0, x.size, 1500)], np.sort(x[-40_000:])

11.4 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
6.96 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


# Warn log1p

In [1]:
import anndata as ad
import numpy as np
from illico.utils.type import scipy_to_nb
from numba import njit
import scanpy as sc

# h1 = ad.read_h5ad('../vcc_h1_904_genes.h5ad').layers["csr"]
h1 = ad.read_h5ad('/Users/remydubois/Documents/repos/vcc/data/competition_support_set/competition_train.h5').X
h1

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 1932554688 stored elements and shape (221273, 18080)>

In [12]:
h1.data.max()

np.float32(7.768956)

In [7]:
@njit(nogil=True, cache=False)
def sampled_max(data: np.ndarray, sample_size: int = 200_000) -> float:
    max_val = -np.inf
    n = data.size
    step = max(1, n // sample_size)
    for i in range(0, n, step):
        if data[i] > max_val:
            max_val = data[i]
    return max_val

sampled_max(h1.data[:1])
%timeit -n 1 -r 1 sampled_max(h1.data)
sampled_max(h1.data)

1.59 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


6.7405195236206055

In [9]:
2e3

2000.0

# Datagen

In [1]:
import anndata as ad

# h1 = ad.read_h5ad('/Users/remydubois/Documents/repos/vcc/data/competition_support_set/competition_train.h5').X.tocsc()
h1

NameError: name 'h1' is not defined

In [7]:
%load_ext autoreload
%autoreload 2
from scipy import sparse
import anndata as ad
# k562 = ad.read_h5ad('/Users/remydubois/Documents/repos/vcc/data/competition_support_set/k562_gwps.h5')
k562 = ad.read_h5ad('/Users/remydubois/Documents/repos/vcc/data/competition_support_set/k562_gwps.h5')
# org_X = k562.X.copy()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
k562.X = sparse.csc_matrix(k562.X)

In [9]:
from illico import asymptotic_wilcoxon
import numpy as np

asymptotic_wilcoxon(k562, is_log1p=False, group_keys="gene", reference_group="non-targeting", n_threads=8, batch_size=256);
# np.testing.assert_allclose(k562.X[:1024], org_X[:1024])

/Users/remydubois/Documents/perso/repos/illico/illico/utils/math.py:150: UserWarning: User indicated is_log1p=False, but estimated data max value is 6.67, Which seems inconsistent. Make sure data is indeed raw counts.
  warnings.warn(
2025-11-25 17:48:50.827 | INFO     | illico.asymptotic_wilcoxon:asymptotic_wilcoxon:64 - Using batch size of 256 for 8 threads and 18080 genes.


  0%|          | 0/3326720 [00:00<?, ?it/s]

In [13]:
from illico import asymptotic_wilcoxon
# k562.X = sparse.csc_matrix(k562.X)
asymptotic_wilcoxon(k562, is_log1p=False, group_keys="gene", reference_group="non-targeting", n_threads=8, batch_size=256);

/Users/remydubois/Documents/perso/repos/illico/illico/utils/math.py:150: UserWarning: User indicated is_log1p=False, but estimated data max value is 6.81, Which seems inconsistent. Make sure data is indeed raw counts.
  warnings.warn(
2025-11-25 17:53:31.824 | INFO     | illico.asymptotic_wilcoxon:asymptotic_wilcoxon:23 - Using batch size of 256 for 8 threads and 18080 genes.


  0%|          | 0/3326720 [00:00<?, ?test/s]

KeyboardInterrupt: 

In [2]:
import numpy as np
from illico.utils.ranking import sort_along_axis
from illico.ovo import csc_ovo_mwu_kernel_over_contiguous_col_chunk, dense_ovo_mwu_kernel_over_contiguous_col_chunk
from illico.utils.groups import encode_and_count_groups
from illico.utils.type import scipy_to_nb

grpc = encode_and_count_groups(k562.obs['gene'].tolist(), ref_group='non-targeting')
# x = k562[k562.obs['gene'] == 'non-targeting'].X[:, :1024]
# sort_along_axis(x[:1], axis=0)
# %timeit -n 1 -r 1 np.sort(x, axis=0)
# %timeit -n 1 -r 1 sort_along_axis(x, axis=0)
# %timeit -n 1 -r 1 sparse.csc_matrix(x)

# chunk = k562.X[:, :1024].copy()
# chunk = np.ones((k562.X.shape[0], 1024), dtype=np.float32)
# for sparsity_rate in np.arange(0., 1., 0.1):
#     if sparsity_rate > 0.0:
#         step = int(1 / sparsity_rate)
#         chunk[::step] = 0
#     print('----- sparsity rate:', sparsity_rate, "avg expression value:", chunk.mean())
#     csc_chunk = scipy_to_nb(sparse.csc_matrix(chunk))
#     csc_ovo_mwu_kernel_over_contiguous_col_chunk(csc_chunk, 0, 1, grpc, is_log1p=False);
#     %timeit -n 1 -r 1 csc_ovo_mwu_kernel_over_contiguous_col_chunk(scipy_to_nb(sparse.csc_matrix(chunk)), 0, 1024, grpc, is_log1p=False);
#     chunk[:] = 1
# dense_ovo_mwu_kernel_over_contiguous_col_chunk(chunk, 0, 1, grpc, is_log1p=False)
# %timeit -n 1 -r 1 dense_ovo_mwu_kernel_over_contiguous_col_chunk(chunk, 0, 1024, grpc, is_log1p=False);
# %timeit -n 1 -r 1 csc_ovo_mwu_kernel_over_contiguous_col_chunk(scipy_to_nb(sparse.csc_matrix(chunk)), 0, 1024, grpc, is_log1p=False);

In [51]:
from illico.ovo.dense_ovo import dense_ovo_mwu_kernel_over_contiguous_col_chunk, dense_ovo_mwu_kernel
from illico.ovo.sparse_ovo import single_group_sparse_ovo_mwu_kernel
ref_indices = grpc.indices[grpc.indptr[grpc.encoded_ref_group]:grpc.indptr[grpc.encoded_ref_group + 1]]
pert_indices = grpc.indices[grpc.indptr[0]:grpc.indptr[1]]
# X_ref = np.sort(k562.X[ref_indices, :1024], axis=0)
# X_pert = np.sort(k562.X[pert_indices, :1024], axis=0)
X_ref = np.ones((len(ref_indices), 1024), dtype=np.float32, order="F")
X_pert = np.ones((len(pert_indices), 1024), dtype=np.float32, order="F")
%timeit -n 1 -r 1 dense_ovo_mwu_kernel(sorted_ref_data=X_ref, sorted_tgt_data=X_pert)
csc_X_ref = scipy_to_nb(sparse.csc_matrix(X_ref))
csc_X_pert = scipy_to_nb(sparse.csc_matrix(X_pert))
%timeit -n 1 -r 1 single_group_sparse_ovo_mwu_kernel(sorted_ref_data=csc_X_ref, sorted_tgt_data=csc_X_pert)

29.1 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
34.7 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [48]:
from illico.utils.ranking import rank_sum_and_ties_from_sorted

f_X_ref = np.asfortranarray(X_ref)
f_X_pert = np.asfortranarray(X_pert)
x, y = f_X_ref[:, 0], f_X_pert[:, 0]
%timeit -n 1 -r 1 rank_sum_and_ties_from_sorted(X_ref[:, 0], X_pert[:, 0])
%timeit -n 1 -r 1 rank_sum_and_ties_from_sorted(f_X_ref[:, 0], f_X_pert[:, 0])
%timeit -n 1 -r 1 rank_sum_and_ties_from_sorted(x, y)
%timeit -n 1 -r 1 rank_sum_and_ties_from_sorted(csc_X_ref.data[0:75328], csc_X_pert.data[0:152])


289 μs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
150 μs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
23.5 μs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
28.2 μs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [47]:
f_X_ref[:, 0].flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : True
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

In [27]:
csc_X_ref.data[0:75328].flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : True
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

In [11]:
# %lprun -u 1e-3 -f single_group_sparse_ovo_mwu_kernel.__wrapped__ single_group_sparse_ovo_mwu_kernel.__wrapped__(sorted_ref_data=csc_X_ref, sorted_tgt_data=csc_X_pert)
single_group_sparse_ovo_mwu_kernel(sorted_ref_data=csc_X_ref, sorted_tgt_data=csc_X_pert)

(array([1., 1., 1., ..., 1., 1., 1.], shape=(1024,)),
 array([5724928., 5724928., 5724928., ..., 5724928., 5724928., 5724928.],
       shape=(1024,)))

In [ ]:
from illico import asymptotic_wilcoxon
asymptotic_wilcoxon(k562, is_log1p=False, group_keys="gene", reference_group="non-targeting", n_threads=1, batch_size=1024)
# k562.obs

/Users/remydubois/Documents/perso/repos/illico/illico/utils/math.py:150: UserWarning: User indicated is_log1p=False, but estimated data max value is 6.67, Which seems inconsistent. Make sure data is indeed raw counts.
  warnings.warn(
2025-11-25 15:15:28.681 | INFO     | illico.asymptotic_wilcoxon:asymptotic_wilcoxon:64 - Using batch size of 1024 for 1 threads and 18080 genes.


  0%|          | 0/18 [00:00<?, ?it/s]

In [10]:

from pdex import parallel_differential_expression
from pdex._single_cell import parallel_differential_expression_vec_wrapper

parallel_differential_expression(k562, groupby_key="gene", reference="non-targeting", num_workers=8)

INFO:pdex._single_cell:Auto-Detected log1p for dataset.
INFO:pdex._single_cell:Log1p status: True
INFO:pdex._single_cell:Precomputing masks for each target gene
Identifying target masks: 100%|██████████| 184/184 [00:00<00:00, 23434.00it/s]
INFO:pdex._single_cell:Precomputing variable indices for each feature
Identifying variable indices: 100%|██████████| 18080/18080 [00:00<00:00, 7538822.58it/s]
INFO:pdex._single_cell:Creating shared memory memory matrix for parallel computing
INFO:pdex._single_cell:Creating generator of all combinations: N=3326720
INFO:pdex._single_cell:Creating generator of all batches: N=33268
INFO:pdex._single_cell:Initializing parallel processing pool
INFO:pdex._single_cell:Processing batches
Processing batches:   1%|▏         | 485/33268 [00:35<50:23, 10.84it/s]  Process SpawnPoolWorker-6:
Process SpawnPoolWorker-5:
Processing batches:   1%|▏         | 486/33268 [00:35<40:25, 13.51it/s]Process SpawnPoolWorker-8:

Process SpawnPoolWorker-2:
Process SpawnPoolWorker

KeyboardInterrupt: 